In [ ]:
# !pip install dask>2.30.0
# !pip install nd2reader
# !pip install pims
# !pip install flowdec
# !pip install aicsimageio==3.3.4


In [22]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt
from flowdec.nb import utils as nbutils 
from flowdec import data as fd_data
# import pims
# import pims
from flowdec import restoration as fd_restoration
from flowdec import data as fd_data
from flowdec import psf as fd_psf
import dask
import dask.array as da
import tifffile as tf
from nd2reader import ND2Reader
# from pims import ND2_Reader 
# import aicsimageio.vendor.omexml as ome
import glob
import time, sys
sys.path.insert(0,'./libraries')
sys.path.insert(0,'./')
from deco_libraries import update_progress, pimsmeta2OMEXML, Nd2meta2OMEXML, observer, init_RL_algo, deconv, depth_divide

In [23]:
#THINGS TO CHANGE!!!!! Always keep /*.nd2 at the end
#For gui make a function with all this input for verification purposes when loading the files
# dirname = "D:/igfp1_caruby5_continue001trigger002.nd2"
# dirname = "D:/JM/20201222caru3/Carubyigfp_F2_Nh4cl_vitc_rutin_BVD002.nd2"
# dirname = "X:/Mamede/JM/2020/20200626/igfp1_caruby5_continue001trigger00*.nd2"
#like this format.D:\Stephanie\20210702F2
dirname = "./*.nd2"
filelist = glob.glob(dirname)
filelist.sort()

# filelist = filelist[2:3]
# filelist.pop()
print(filelist)
# print(filelist[1])

#Load the first file for reference and what's inside the file
frames = ND2Reader(filelist[0])
print(frames.sizes)
#Grab metadata
metadata = frames.metadata
print(metadata['channels'])

#If you don't have enough VRAM divide the XY to do the deconvolution that is automatically assembled after
xdivide = 1
ydivide = 1
depthdivide = depth_divide(xdivide, ydivide)
frames

['./20210611_E1_cGas_1A11PQBC1_241D_7131.nd2']
{'x': 1024, 'y': 1024, 'c': 4, 't': 1, 'z': 66, 'v': 10}
['pqbp1-AF647', 'mRuby3', 'Igfp', 'Cgas-DY405']


<FramesSequenceND>
Axes: 6
Axis 'x' size: 1024
Axis 'y' size: 1024
Axis 'c' size: 4
Axis 't' size: 1
Axis 'z' size: 66
Axis 'v' size: 10
Pixel Datatype: <class 'numpy.float64'>

In [24]:

metadata['pixel_microns']


0.14444444444444468

In [25]:
xml = Nd2meta2OMEXML(frames,time_offset=0,maxT=None,verbose=False)
xml.image().Pixels.PhysicalSizeZ

0.2

In [59]:
#Need to make this a big more automatic, but indexing all the files and iterations is not easy
#In the version where the PSF is estimated by flowdec is easier because everything was set to automatic

# PSFdir="/content/PSF"
# psf647 = tf.imread(PSFdir+"/AF647_0.3_EMCCD.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.3_EMCCD.tif')
# psfgfp =  tf.imread(PSFdir+'/green_0.3_EMCCD.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI_0.3_EMCCD.tif')
# psf647 = tf.imread(PSFdir+"/AF647_0.5_EMCCD.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.5_EMCCD.tif')
# psfgfp =  tf.imread(PSFdir+'/green_0.5_EMCCD.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI_0.5_EMCCD.tif')
# psf647 = tf.imread(PSFdir+"/AF647_0.5.tiff")
# psfruby =  tf.imread(PSFdir+'/mRuby3_0.5.tiff')
# psfgfp =  tf.imread(PSFdir+'/green_0.5.tiff')
# psfdapi =  tf.imread(PSFdir+'/DAPI.tif')
# psf647 = tf.imread(PSFdir+"/AF647.tif")
# psfruby =  tf.imread(PSFdir+'/mRuby3.tif')
# psfgfp =  tf.imread(PSFdir+'/green.tif')
# psfdapi =  tf.imread(PSFdir+'/DAPI.tif')
# psfruby = psf647
# psfgfp = psf647
# psfdapi = psf647
    #na = meta['NA']
na = 1.4
#ra = meta['refractive_index1']
ra = 1.5156
z = int(frames.sizes['z'])-1
# z = 6
#     x = frames.sizes['x'] // 2
#     y = frames.sizes['y'] // 2
#     z = 26
x = 256
y = 256
res_z = xml.image().Pixels.PhysicalSizeZ
res_xy =  metadata['pixel_microns']
distance_cover = -2
##Make better with automatic wavelength from metadata and a loop and a dictionary with the PSFs
psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.670 , size_z=  z, size_x=  x , size_y=  y,
    m =60, 
     ns =  1.333,
     tg =  170, tg0 = 170,
     ni0 =  ra, ti0 = 130,
     ni =  ra, 
     res_lateral =  res_xy , res_axial = res_z, 
     pz = distance_cover 
)
psf647 = psf.generate()    

psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.620, size_z=  z, size_x=  x , size_y=  y,
    m =60, 
     ns =  1.333,
     tg =  170, tg0 = 170,
     ni0 =  ra, ti0 = 130,
     ni =  ra, 
     res_lateral =  res_xy , res_axial = res_z, 
     pz = distance_cover 
)
psfruby = psf.generate()

psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.535, size_z=  z, size_x=  x , size_y=  y,
    m =60, 
     ns =  1.333,
     tg =  170, tg0 = 170,
     ni0 =  ra, ti0 = 130,
     ni =  ra, 
     res_lateral =  res_xy , res_axial = res_z, 
     pz = distance_cover 
)
psfgfp = psf.generate()

psf = fd_psf.GibsonLanni(
    na = na, wavelength= 0.420, size_z=  z, size_x=  x , size_y=  y,
    m =60, 
     ns =  1.333,
     tg =  170, tg0 = 170,
     ni0 =  ra, ti0 = 130,
     ni =  ra, 
     res_lateral =  res_xy , res_axial = res_z, 
     pz = distance_cover 
)
psfdapi = psf.generate()  
psf = psfruby
#                 )
#Clip Top and bottom, I found that if the PSF is calculated with distance from coverslip, it is flipped from Fiji
#As in, it is Top to Bottom, I acquire the images Bottom to Top
# remove = 13
# remove = 14
# psf647 = np.flip(psf647,
# #                  axis=0
#                 )[remove:-remove]

# psfgfp = np.flip(psfgfp,
# #                  axis=0
#                 )[remove:-remove]
# psfdapi = np.flip(psfdapi,
# #                   axis=0
#                  )[remove:-remove]
# psfruby = np.flip(psfruby,
# #                   axis=0
#                  )[remove:-remove]
# psf = tf.imread('./Rush/PSF/PSFGL.tif')
print(psf.shape)
# print(psfruby[22:-22,...].shape)

(65, 256, 256)


In [60]:
psf_1 = np.flip(psf[0:32,...],axis=0)
# tf.imsave('./Rush/PSF/pfsflowdec2.tiff',psfgfp)
# psf = psfruby[22:-22,...]
# psf = psfruby
# psf = tf.imread('./Rush/PSF/PSF_GL.tif')
# print(psf[28:-28,...].shape)

In [61]:
psf[33:65] = psf_1

In [54]:
algo =init_RL_algo(psfgfp.ndim
                   ,pad_mode='2357',pad_min=(2,24,24)
#                    ,pad_mode='log2',pad_min=(2,24,24)
                  )
# from flowdec import restoration as fd_restoration
# algo = fd_restoration.RichardsonLucyDeconvolver(n_dims=psfgfp.ndim
#                                                     # , pad_mode=pad_mode
#                                                     # ,pad_min=pad_min
#     #                                                     ,observer_fn=observer
#                                                     #,real_domain_fft=True
#                                                     #,device='/cpu:0'
#                                                    ).initialize()

print(filelist)
# fname2 = filelist[1]
fname1 = filelist[0]
# filelist.pop(0)

print(filelist)

['./20210611_E1_cGas_1A11PQBC1_241D_7131.nd2']
['./20210611_E1_cGas_1A11PQBC1_241D_7131.nd2']


In [55]:
from deco_libraries import update_progress, pimsmeta2OMEXML, Nd2meta2OMEXML, observer, init_RL_algo, deconv, depth_divide

In [56]:

# frames.parser._raw_metadata.image_metadata
def get_deltaT(filename1,filename2):
    
    import pathlib
#     frames = ND2Reader(filelist[0])
#     a = frames.metadata['date']
#     frames = ND2Reader(filelist[1])
#     b = frames.metadata['date']
    fname1 = pathlib.Path(filename1)
    fname2 = pathlib.Path(filename2)
    import datetime
    mtime2 = datetime.datetime.fromtimestamp(fname2.stat().st_ctime)
    mtime1 = datetime.datetime.fromtimestamp(fname1.stat().st_ctime)
#     print(mtime2,mtime1)
#     print(fname2.stat().st_ctime - fname1.stat().st_ctime)
    return fname2.stat().st_ctime - fname1.stat().st_ctime
    # time_offset = 71146.03463482857
# fname2 = filelist[1]
# fname1 = filelist[0]   
# print(get_deltaT(fname1,fname2)/60)
# print(get_deltaT(fname1,fname2)/60)

# time_offset = get_deltaT(fname1,fname2)

In [57]:
# filelist.pop(0)
filelist

['./20210611_E1_cGas_1A11PQBC1_241D_7131.nd2']

In [62]:
#these are the settings you need to look at
n_iters = 100 # keep this one always at 100

savePRJ = True # if you want the projection Z have it at True, if not, False
saveStack = True # if you want the Z-stack have it at True, if not, False 


# time_offset = 0
# maxT = 25
#do not worry about these ones
maxT = None

#If it's a broken experiment, you need to do concat Time True, 
#so that it makes the 0 from file #2 to be the diference between the files
concattime = False

for fname in filelist:
    #to add the real time of experiment when we have to split it into different files
    if concattime:
#         time_offset = get_deltaT(filelist[0],fname)
        time_offset = get_deltaT(fname1,fname)
#         time_offset = 72424
#         time_offset = 72424
    else: time_offset = 0
    frames =  ND2Reader(fname)
    frames.iter_axes = 't'  # 't' is the default already
    frames.bundle_axes = 'zyx'  # when 'z' is available, this will be default

    channels = frames.metadata['channels']
    metadata = frames.metadata
    
    if saveStack:
        xml = Nd2meta2OMEXML(frames,time_offset=time_offset,maxT=maxT,verbose=False)
    if savePRJ:
        prjxml = Nd2meta2OMEXML(frames, project=True,time_offset=time_offset,maxT=maxT,verbose=False)

    chunk_size=(frames.sizes['z'],frames.sizes['y']//ydivide,frames.sizes['x']//xdivide)

    # frames.iter_axes = 'v'
    # for frame in frames:
#     for visit in range(22,25):
#         frames.sizes['v']):
#     for visit in range(frames.sizes['v']):
    for visit in [0]:

        frames.default_coords['v'] = visit
        stackfile = fname[:-4]+"_v"+str(visit+1).zfill(2)+'.Glani.ome.tiff'
        prjfile = fname[:-4]+"_v"+str(visit+1).zfill(2)+'_PRJ.Glani.ome.tiff'
        if saveStack:
            tifstack = tf.TiffWriter(stackfile, bigtiff=False,imagej=False)
            firststk = True
        if savePRJ:
            tifprj = tf.TiffWriter(prjfile, bigtiff=False, imagej=False)
            first = True
        if maxT == None:
            counterT = frames.sizes['t']
        else: counterT = maxT
# +
        for time in range(counterT):
#         for time in range(25):

#             res = np.zeros(
#                 (frames.sizes['z'],frames.sizes['c']
#                  ,frames.sizes['y'],frames.sizes['x'])
#                 , dtype=np.float32)
    
            res = np.zeros(
                (frames.sizes['z'],frames.sizes['c']
                 ,frames.sizes['y'],frames.sizes['x'])
                , dtype=np.float32)

            for i in range(frames.sizes['c']):
#             for i in range(3, frames.sizes['c']):
                frames.default_coords['c'] = i
#                     print('Channel:',metadata['channels'][i])
                ch = metadata['channels'][i]
                # ['a647', 'DAPI', 'GFP']
                # ['FITC', 'mRuby3', 'miRFP670']
                # ['FRET-gYFP-dsRED', 'mRuby', 'GFP', 'DAPI']
                # ['a647', 'DAPI', 'GFP']
#                 if ch == 'DAPI' or ch  == '470 nm':
#                     psf = psfdapi
#                 elif ch == 'miRFP670' or ch == 'a647' or ch == 'mirfp670' or ch == 'farRED-EM' or ch == '640 nm':
#                     psf = psf647
#                 elif ch == 'GFP' or ch == 'FITC'  or ch == 'GREEN-EM' or ch == 'FITC (Em)':
#                     psf = psfgfp
#                 elif ch == 'mRuby3' or ch == 'mRuby' or ch ==  '555 nm' or ch == 'RED-EM':
#                     psf = psfruby
#                 elif ch == 'FRET-gYFP-dsRED':
#                     psf = psfruby
#                 else: 
                psf = psf

                arr = da.from_array(frames[time]           
                                    , chunks=chunk_size)

                res[:,i,:,:] = da.map_overlap(deconv,arr, depth  = depthdivide ,boundary='reflect',
                                    dtype='float32', algo=algo,
                                    psf=psf, iters=n_iters, use_ram=False
                                             ).compute(num_workers=1)

                # If we need to swap Channel and Z in the future
                #             img5d = np.swapaxes(img5d,0,1)

                update_progress( (visit+1) * (time+1) / (frames.sizes['v']* counterT) )
                print("Visit Point: {}/{} \t Time:{}/{} \t Channel:{}/{}".format(
                        visit+1, frames.sizes['v']
                        ,time+1,  counterT
                        , i+1, frames.sizes['c']))
# Make Function
            if saveStack:

                if firststk:
                    tifstack.save(res.astype(np.uint16),
                        compress='ZLIB',
                        description = xml.to_xml(),
                        photometric='minisblack',
                        metadata= None,
                        contiguous=False
                        )
                    firststk = False
                    print('blup')
                else:
                    tifstack.save(res.astype(np.uint16),
                        compress='ZLIB',
#                         description = xml.to_xml(),
#                         datetime= True,
                        photometric='minisblack',
                        metadata= None,
                        contiguous=False
                        )
            if savePRJ:
                if first:
                    tifprj.save(res.astype(np.uint16).max(axis=0),
                        compress='ZLIB',
                        description = prjxml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False
                        )
                    first = False
                else:                  
                    tifprj.save(res.astype(np.uint16).max(axis=0),
                        compress='ZLIB',
#                         description = xml.to_xml(),
                        photometric='minisblack',
                        #, datetime= True
                        metadata= None,
                        contiguous=False,
                        )
        if saveStack:
            tifstack.close()
        if savePRJ:
            tifprj.close()

    update_progress(1)

Progress: [####################] 100.0%


In [ ]:
print('done')
from numba import cuda
cuda.select_device(0)
cuda.close()
#the memory was released here!
cuda.select_device(0)

In [ ]:
!nvidia-smi

In [ ]:
frames =  ND2Reader(fname)

In [ ]:
import tensorflow as tenf
tenf.config.list_physical_devices('GPU')


In [ ]:
import flowdec
flowdec.__version__                                                                                

In [ ]:
import dask
dask.__version__